In [212]:
import pandas as pd
import numpy as np
from io import BytesIO
import requests
from urllib.parse import urlencode
import urllib
from requests.auth import HTTPBasicAuth
from requests.exceptions import ChunkedEncodingError
import os
import json
import yadisk
from datetime import datetime, date, timedelta
import locale
from time import sleep
import shutil
import gc
import turbodbc
from turbodbc import connect
import gc
from pandas.api.types import is_string_dtype
import numpy as np
from sqlalchemy import create_engine
import pyodbc
import warnings


pd.options.mode.chained_assignment = None


# параметры подключения к нашей БД, в которую сохранить итоговые таблицы
server_local = 'REGRU-54856\MSSQLSERVER01'
database_local = 'custom_tables_NL_2208' #'custom_tables_NL_2208' - БД для тестовой загрузки / 'NL_db_27082024' - БД для дашбодра НЛ
conn_str_local = 'DRIVER={SQL Server};SERVER='+server_local+';DATABASE='+database_local+';'

# указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\32_NL_Yandex_Disk'
file_name = 'yandex_token.txt'

# сохраняем токен в отдельную переменную
yandex_token = open(os.path.join(file_path, file_name), encoding='utf-8').read()
# формируем заголовки для дальнейших запросов
headers = {
    'Content-Type': 'application/json', 
    'Accept': 'application/json', 
    'Authorization': f'OAuth {yandex_token}'
}

# указываем путь к основной папке, в которой храняться папки с флайтами
main_folder = '/NL_отчеты_для_дашборда'

# УРЛ для операций над опубликованными папками и файлами на Яндекс Диске
base_public_url = 'https://cloud-api.yandex.net/v1/disk/public/resources?'  
public_key = 'https://disk.yandex.ru/d/2iYr8zMm9Qgeug' # обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета

download_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?' # УРЛ для скачивания эксель файлов из Яндекс Диска
delete_url = 'https://cloud-api.yandex.net/v1/disk/resources?' # УРЛ для удаления папок и файлов на Яндекс Диске

final_url = base_public_url + urlencode(dict(public_key=public_key))  # Формируем УРЛ для получения списка папок и файлов
res = requests.get(final_url)  # Отправлем запрос на Яндекс диск, чтобы получить название папок и public key для скачивания файлов

print(res.status_code)

yandex_folders = res.json() # парсим ответ

200


In [213]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)

# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)

pd.set_option('display.max_rows', None)

pd.options.mode.chained_assignment = None

In [214]:
# Создаем функцию для получения содержимого Яндекс диска
#  на входе принимаем 
# - адрес запроса для скачивания файла
# - public key, который мы получаем из ответа Яндекс при запросе информации по конкретной папке
# - адрес папки
# - лимит файлов (максимальное кол-во 80)
def get_yandex_disk_responce(request_url, public_key, folder_path, limit=80):
    res_url = request_url + urlencode(dict(public_key=public_key, path=folder_path, limit=limit)) # формируем строку запроса
    res = requests.get(res_url) # отправляем запрос на сервер
    yandex_responce = res.json() # получаем ответ и преобразуем его в json

    return yandex_responce

In [215]:
public_key = yandex_folders['public_key']  # из ответа Яндекс забираем public_key, чтобы использовать его для скачивания файлов

for i in range(len(yandex_folders['_embedded']['items'])): # через цикл проходим по ответу Яндекса и забираем названия вложенных папок
    file_type = yandex_folders['_embedded']['items'][i]['type']
    if file_type=='dir':   # если находим файлы с типом dir (папка), то забираем путь к этой папке
        folder_path = yandex_folders['_embedded']['items'][i]['path']
        print(folder_path)
        yandex_responce = get_yandex_disk_responce(base_public_url, public_key, folder_path) # отправляем запрос, чтобы получить содержимое папки

        # Через цикл проходим по папке с файлами
        # Нас интересуют файлы эксель. Причем каждая экселька будет парситься по своему, т.к. они относятся к разным рекламным площадкам
        
        # Проходим через цикл по содержимому папки (отдельный флайт)
        for i in range(len(yandex_responce['_embedded']['items'])):
            file_info = yandex_responce['_embedded']['items'][i]
            if file_info['type']=='file':  # если документ является фалйом(не папкой или изображением), то забираем его название 
                file_name = file_info['name'] # сохраняем название файла
                if 'xls' in file_name: # еслит тип файла является xlsx, то уберем расширение и будем его использовать в качесвте названия отчета
                    file_path = file_info['path']
                    
                    report_name = '.'.join(file_name.split('.')[:-1]) # убираем .xlsx из названия файла
                    print(report_name)
                    res_file_link = get_yandex_disk_responce(download_url, public_key, file_path) # получаем ссылку на скачивание отчета
                    download_response = requests.get(res_file_link['href'])

/01_new_sources_files
/02_regular_reports
turbotarget_banner


In [216]:
data_link = download_response.content

In [217]:
sheet_names = pd.ExcelFile(BytesIO(data_link))

In [218]:
sheet_names.sheet_names

['Август+сентябрь', 'Август', 'по креативам']

In [219]:
report_type = 'test'

In [252]:
tmp_video_dict = {}

# создаем список с названиями полей
cols_range = 'A:F'

df = pd.read_excel(BytesIO(data_link), sheet_name='по креативам', header=None, usecols=cols_range)
# df = df.fillna('')
start_index_list = list(df[df[0]=='Дата'].index) # сохраняем в список индексы строк начала каждой таблицы с данными
end_index_list = list(df[df[0]=='Итого'].index) # сохраняем в список индексы строк окончания каждой таблицы с данными

df.columns = df.iloc[start_index_list[0]+1].str.lower().str.strip().str.replace('\n', ' ') # забираем название полей из файла
df = df.rename(columns={np.nan: 'date', 'показы': 'impressions', 'клики': 'clicks', 'бюджет': 'budget', 'охват': 'reach'})
df = df[['date', 'impressions', 'clicks', 'budget', 'reach']]

for i in range(len(start_index_list)):
    start_index = start_index_list[i] # получаем индкс начала конкретной таблицы
    end_index =  end_index_list[i]  # получаем индкс окончания конкретной таблицы

    name = str(df['date'][start_index-1]).lower().strip() # забираем название креатива, к которому относится статистика в таблице

    df_tmp = df.iloc[start_index+2:end_index] # оставляем строки с данными, которые нам нужны
    df_tmp = df_tmp.fillna('')
    df_tmp = df_tmp[df_tmp['impressions'] != '']
    df_tmp['product'] = name
    df_tmp['source'] = 'turbotarget' #добавляем название источника
    df_tmp['format_type'] = 'banner' # добавляем статичое поле с название Типа формата рекламы (Видео/Баннер)
    df_tmp['date'] = pd.to_datetime(df_tmp['date']).dt.date  # приводим в формат даты
    
    df_tmp['report_type'] = report_type
    tmp_video_dict[name] = df_tmp

In [253]:
tmp_video_dict

{'creo_3': 3         date impressions clicks budget  reach product       source  \
 4   2024-08-28       13689     29   2175   6426  creo_3  turbotarget   
 5   2024-08-29       16003     22   1650   7340  creo_3  turbotarget   
 6   2024-08-30       72374    116   8700  32166  creo_3  turbotarget   
 7   2024-08-31       86242    133   9975  36236  creo_3  turbotarget   
 8   2024-09-01       77472    148  11100  31621  creo_3  turbotarget   
 9   2024-09-02      120342    135  10125  47192  creo_3  turbotarget   
 10  2024-09-03      114459    137  10275  42708  creo_3  turbotarget   
 11  2024-09-04       90347    138  10350  32817  creo_3  turbotarget   
 12  2024-09-05      107427     86   6450  34103  creo_3  turbotarget   
 13  2024-09-06      100738     76   5700  33028  creo_3  turbotarget   
 14  2024-09-07       55010     64   4800  17745  creo_3  turbotarget   
 15  2024-09-08       54807     69   5175  17679  creo_3  turbotarget   
 16  2024-09-09       61312     84   6300

In [248]:
df

3,date,impressions,clicks,budget,reach
0,NaN,NaN,NaN,NaN,NaN
1,creo_3,NaN,NaN,NaN,NaN
2,Дата,Статистика Turbotarget,NaN,NaN,NaN
3,NaN,Показы,Клики,Бюджет,Охват
4,2024-08-28 00:00:00,13689,29,2175,6426
5,2024-08-29 00:00:00,16003,22,1650,7340
6,2024-08-30 00:00:00,72374,116,8700,32166
7,2024-08-31 00:00:00,86242,133,9975,36236
8,2024-09-01 00:00:00,77472,148,11100,31621
9,2024-09-02 00:00:00,120342,135,10125,47192


In [ ]:
# источник turbotarget
# тип отчета Баннерная реклама
# Функция для обработки статистики по дням для обычных размещений 
# Статистика по креативам находится на листе - по креативам 
# название креатива находится в строке ПЕРЕД таблицей НАД полем дата
# шапка заголовка таблицы состоит из 2-х совмещенных строк
# каждая таблица заканчивается строкой Итого

def get_turbotarget_banner_report(data_link, report_type):
    tmp_video_dict = {}

    # создаем список с названиями полей
    cols_range = 'A:F'
    cols_name = ['date', 'impressions', 'clicks', 'ctr', 'budget', 'reach']
    df = pd.read_excel(BytesIO(data_link), sheet_name='по креативам', header=None, usecols=cols_range, names=cols_name)

    df = df[['date', 'impressions', 'clicks', 'budget', 'reach']]
    start_index_list = list(df[df['date']=='Дата'].index) # сохраняем в список индексы строк начала каждой таблицы с данными
    end_index_list = list(df[df['date']=='Итого'].index) # сохраняем в список индексы строк окончания каждой таблицы с данными
    
    
    for i in range(len(start_index_list)):
        start_index = start_index_list[i] # получаем индкс начала конкретной таблицы
        end_index =  end_index_list[i]  # получаем индкс окончания конкретной таблицы
    
        name = df['date'][start_index-1] # забираем название креатива, к которому относится статистика в таблице

        df_tmp = df.iloc[start_index+2:end_index] # оставляем строки с данными, которые нам нужны
        df_tmp = df_tmp.fillna('')
        df_tmp = df_tmp[df_tmp['impressions'] != '']
        df_tmp['product'] = name
        df_tmp['source'] = 'turbotarget' #добавляем название источника
        df_tmp['format_type'] = 'banner' # добавляем статичое поле с название Типа формата рекламы (Видео/Баннер)
        df_tmp['date'] = pd.to_datetime(df_tmp['date']).dt.date  # приводим в формат даты
        df_tmp['product'] = df_tmp['product'].str.lower().str.strip()
        df_tmp['report_type'] = report_type
        tmp_video_dict[name] = df_tmp

    return pd.concat(tmp_video_dict, ignore_index=True)